<a href="https://colab.research.google.com/github/harshakoneru98/city_watch/blob/main/modelling/Crime_Rate_Forecasting_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
import pandas as pd
import json
import tqdm
import seaborn as sns
import numpy as np
import datetime
import tqdm
from sklearn.preprocessing import MinMaxScaler
from keras.layers import LSTM, Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
ROOT_PATH = "/content/drive/MyDrive/DSCI 560/Datasets/Cleaned_data/"

In [17]:
crimes = pd.read_csv(f"{ROOT_PATH}/crime_date_aligned.csv")
crime_by_date_zipcode = crimes.groupby(["date_occ", "zip_code"]).agg({ "dr_no": "count"}).reset_index()

In [18]:
# Convert the 'date' column to datetime format
crime_by_date_zipcode['date'] = pd.to_datetime(crime_by_date_zipcode['date_occ'])

# Create a new column for the day of the week
crime_by_date_zipcode['day_of_week'] = crime_by_date_zipcode['date'].dt.dayofweek

# Create a new column for the week of the year
crime_by_date_zipcode['week_of_year'] = crime_by_date_zipcode['date'].dt.weekofyear

# Create a new column for the day of the month
crime_by_date_zipcode['day_of_month'] = crime_by_date_zipcode['date'].dt.day

# Create a new column for the month of the year
crime_by_date_zipcode['month_of_year'] = crime_by_date_zipcode['date'].dt.month

# Create a new column for the year
crime_by_date_zipcode['year'] = crime_by_date_zipcode['date'].dt.year

crime_by_date_zipcode['previous_7_days'] = crime_by_date_zipcode.set_index('date').groupby('zip_code')['dr_no'].rolling(window="7D").count().values

<ipython-input-18-0387295c068b>:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  crime_by_date_zipcode['week_of_year'] = crime_by_date_zipcode['date'].dt.weekofyear


In [21]:
zip_crime_df = crime_by_date_zipcode.groupby('zip_code')['previous_7_days'].mean().reset_index()
zip_crime_map = {k:v for k,v in zip(zip_crime_df['zip_code'], zip_crime_df['previous_7_days'])}
crime_by_date_zipcode['zipcode_avg'] = crime_by_date_zipcode.zip_code.map(lambda x: zip_crime_map[x])

In [93]:
OUTPUT_COLUMN = 'previous_7_days'
INPUT_COLUMNS = ['day_of_week', 'week_of_year', 'day_of_month', 'month_of_year', 'year', 'zipcode_avg']
WINDOW_LENGTH = 30
train_data = crime_by_date_zipcode[crime_by_date_zipcode['date'] <= datetime.datetime(2022, 9, 30)][INPUT_COLUMNS + [OUTPUT_COLUMN]]
test_data = crime_by_date_zipcode[crime_by_date_zipcode['date'] > datetime.datetime(2022, 9, 30)][INPUT_COLUMNS + [OUTPUT_COLUMN]]
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(train_data[[OUTPUT_COLUMN]])

MinMaxScaler()

In [94]:
def create_dataset(dataset, time_steps=1):
    X, Y = [], []
    dataset = dataset.to_numpy()
    dataset_x = dataset[:,:-1]
    dataset_y = scaler.transform(dataset[:,-2:-1])
    for i in tqdm.tqdm(range(len(dataset) - time_steps - 1)):
        X.append(dataset_x[i:(i + time_steps), :-1])
        Y.append(dataset_y[i:(i + time_steps), -1])
    return np.array(X), np.array(Y)

In [95]:
train_X, train_Y = create_dataset(train_data, WINDOW_LENGTH)

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
100%|██████████| 378290/378290 [00:00<00:00, 916384.15it/s]


In [96]:
test_X, test_Y = create_dataset(test_data, WINDOW_LENGTH)

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
100%|██████████| 9669/9669 [00:00<00:00, 609680.47it/s]


In [97]:
train_X.shape, train_Y.shape, test_X.shape, test_Y.shape

((378290, 30, 5), (378290, 30), (9669, 30, 5), (9669, 30))

In [116]:
# defining model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dense(units=1))

In [117]:
early_stopping_monitor = EarlyStopping(patience = 10)
model.compile(optimizer=Adam(lr=0.001), loss='mean_squared_error')
model.fit(train_X, train_Y, validation_split = 0.1, epochs=10, batch_size=64, callbacks=[early_stopping_monitor])

Epoch 1/10
5320/5320 [==============================] - 44s 8ms/step - loss: 8.5320e-04 - val_loss: 3.2381e-05
Epoch 2/10
5320/5320 [==============================] - 39s 7ms/step - loss: 2.7020e-05 - val_loss: 1.6962e-05
Epoch 3/10
5320/5320 [==============================] - 41s 8ms/step - loss: 2.4986e-05 - val_loss: 1.7012e-05
Epoch 4/10
5320/5320 [==============================] - 41s 8ms/step - loss: 2.4110e-05 - val_loss: 2.0001e-05


In [119]:
predictions = model.predict(test_X, batch_size=128)

76/76 [==============================] - 0s 4ms/step


In [120]:
predictions_inv = scaler.inverse_transform(predictions.reshape((predictions.shape[1] * predictions.shape[0], predictions.shape[2])))
test_Y_inv = scaler.inverse_transform(np.expand_dims(test_Y, axis=-1).reshape((test_Y.shape[1] * test_Y.shape[0], 1)))
mse = np.mean((predictions_inv - test_Y_inv)**2)
print('Root Mean Squared Error:', np.sqrt(mse))

Root Mean Squared Error: 0.02476514068856018
